In [33]:
import pandas as pd
import sklearn 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn import feature_selection
import seaborn as sns
import openpyxl

In [40]:
janela = 'moedas'
book_name='Book1'
df = pd.read_csv("P:\\ciencia_de_dados\\Correlacao_de_fundos\\Bases\\"  + book_name + "_" + janela + ".csv",delimiter=',',decimal='.',parse_dates=True)
df = df.dropna().drop_duplicates().rename(columns={'Unnamed: 0':'data'})
df.head()

,data,CRY Index,JPEIGLSP Index,BRL Curncy,CAD Curncy,AUD Curncy,DXY Index
2,2011-09-30,298.15,464.96,1.8793,1.0503,0.9662,78.553
3,2011-10-31,319.84,391.73,1.7157,1.0008,1.053,76.166
4,2011-11-30,313.82,413.25,1.8085,1.0174,1.0283,78.384
5,2011-12-30,305.3,426.34,1.8668,1.0213,1.0209,80.178
6,2012-01-31,312.31,412.23,1.7468,1.0026,1.0621,79.288


In [41]:
df = df.melt(id_vars='data', value_vars=list(df.columns[1:]), value_name="FinancialPrice",var_name='Product')

In [42]:
#Calculo das colunas temporais
df['data']=pd.to_datetime(df['data'],format = '%Y-%m-%d')
df['MesAno'] = df['data'].dt.strftime('%m-%Y')
df['Mes_nomial']=df['data'].dt.strftime('%b')
df['Mes']=df['data'].dt.strftime('%m').astype(int)
df['Ano']=df['data'].dt.strftime('%Y').astype(int)

In [43]:
df

,data,Product,FinancialPrice,MesAno,Mes_nomial,Mes,Ano
0,2011-09-30,CRY Index,298.15,09-2011,Sep,9,2011
1,2011-10-31,CRY Index,319.84,10-2011,Oct,10,2011
2,2011-11-30,CRY Index,313.82,11-2011,Nov,11,2011
3,2011-12-30,CRY Index,305.3,12-2011,Dec,12,2011
4,2012-01-31,CRY Index,312.31,01-2012,Jan,1,2012
...,...,...,...,...,...,...,...
721,2021-05-31,DXY Index,89.829,05-2021,May,5,2021
722,2021-06-30,DXY Index,92.436,06-2021,Jun,6,2021
723,2021-07-30,DXY Index,92.174,07-2021,Jul,7,2021
724,2021-08-31,DXY Index,92.626,08-2021,Aug,8,2021


In [44]:
if janela == 'diaria':
    df = df.groupby(['Product','MesAno']).agg(['max'])
    df.columns = df.columns.droplevel(1)
    df = df1.reset_index()
    df.head()

In [45]:
df.sort_values(by=["Product",'data'],ascending=False,inplace=True)

In [46]:
df

,data,Product,FinancialPrice,MesAno,Mes_nomial,Mes,Ano
241,2021-09-30,JPEIGLSP Index,324.37,09-2021,Sep,9,2021
240,2021-08-31,JPEIGLSP Index,313.01,08-2021,Aug,8,2021
239,2021-07-30,JPEIGLSP Index,325.02,07-2021,Jul,7,2021
238,2021-06-30,JPEIGLSP Index,312.5,06-2021,Jun,6,2021
237,2021-05-31,JPEIGLSP Index,306.69,05-2021,May,5,2021
...,...,...,...,...,...,...,...
488,2012-01-31,AUD Curncy,1.0621,01-2012,Jan,1,2012
487,2011-12-30,AUD Curncy,1.0209,12-2011,Dec,12,2011
486,2011-11-30,AUD Curncy,1.0283,11-2011,Nov,11,2011
485,2011-10-31,AUD Curncy,1.053,10-2011,Oct,10,2011


In [47]:
#Cria a coluna retorno e seta NaN na primeira linha de cada fundo
df['FinancialPrice']=pd.to_numeric(df["FinancialPrice"], downcast="float")
df["Retorno_1"] = (df.FinancialPrice.shift())/(df.FinancialPrice)
df.loc[df.drop_duplicates('Product').index.values,'Retorno_1'] = np.NaN
df['Retorno'] = df.Retorno_1.shift(periods=-1)
df['Retorno'] = (df["Retorno_1"]-1)

In [48]:
df.to_csv('P:\\ciencia_de_dados\\Correlacao_de_fundos\\Desenvolvimento\\1DataPrep\\base_'+ book_name + '.csv',sep =';',decimal=',')